# Morhing Two Images

In [6]:
import dlib
import cv2
import numpy as np

In [5]:
def convert_to_numpy_array(shape, dtype='int'):
    cordinates = np.zeros((shape.num_parts,2),dtype=dtype)
    for i in range(shape.num_parts):
        cordinates[i] = (shape.part(i).x,shape.part(i).y)
    return cordinates

In [4]:
def get_landmark_points(image,landmark_points):
    face_detector = dlib.get_frontal_face_detector()
    detected_face = face_detector(image,1)
    for i,d in enumerate(detected_face):
        shape = landmark_points(image,d)
        points = convert_to_numpy_array(shape)
        points_list = points.tolist()
    return points_list
    

In [3]:
def applyAffineTransform(src, srcTri, dstTri, size) :
    
    # Given a pair of triangles, find the affine transform.
    warpMat = cv2.getAffineTransform( np.float32(srcTri), np.float32(dstTri) )
    
    # Apply the Affine Transform just found to the src image
    dst = cv2.warpAffine( src, warpMat, (size[0], size[1]), None, flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REFLECT_101 )

    return dst

In [2]:
def morphTriangle(img1, img2, img, t1, t2, t, alpha) :

    # Find bounding rectangle for each triangle
    r1 = cv2.boundingRect(np.float32([t1]))
    r2 = cv2.boundingRect(np.float32([t2]))
    r = cv2.boundingRect(np.float32([t]))


    # Offset points by left top corner of the respective rectangles
    t1Rect = []
    t2Rect = []
    tRect = []


    for i in range(0, 3):
        tRect.append(((t[i][0] - r[0]),(t[i][1] - r[1])))
        t1Rect.append(((t1[i][0] - r1[0]),(t1[i][1] - r1[1])))
        t2Rect.append(((t2[i][0] - r2[0]),(t2[i][1] - r2[1])))


    # Get mask by filling triangle
    mask = np.zeros((r[3], r[2], 3), dtype = np.float32)
    cv2.fillConvexPoly(mask, np.int32(tRect), (1.0, 1.0, 1.0), 16, 0);

    # Apply warpImage to small rectangular patches
    img1Rect = img1[r1[1]:r1[1] + r1[3], r1[0]:r1[0] + r1[2]]
    img2Rect = img2[r2[1]:r2[1] + r2[3], r2[0]:r2[0] + r2[2]]

    size = (r[2], r[3])
    warpImage1 = applyAffineTransform(img1Rect, t1Rect, tRect, size)
    warpImage2 = applyAffineTransform(img2Rect, t2Rect, tRect, size)

    # Alpha blend rectangular patches
    imgRect = (1.0 - alpha) * warpImage1 + alpha * warpImage2

    # Copy triangular region of the rectangular patch to the output image
    img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] = img[r[1]:r[1]+r[3], r[0]:r[0]+r[2]] * ( 1 - mask ) + imgRect * mask

In [12]:

alpha = 0.4

# Base image for morphing
base_image = cv2.imread("files/brunomars.jpg")

# Image to be overlayed
overlay_image = cv2.imread("files/dakotajohnson.jpg")

# Base image 68 landmark points
base_landmark_points = "files/brunomarspoints.txt"

# Overlay image 68 landmark points
overlay_landmark_points = "files/dakotajohnsonpoints.txt"

# Base image delaunay points
base_delaunay_points = "files/brunotxtp1p2p3.txt"

frontal_face_detector = dlib.get_frontal_face_detector()

landmark_points_detector = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

base_image_points = get_landmark_points(base_image,landmark_points_detector)

overlay_image_points = get_landmark_points(overlay_image,landmark_points_detector)

points = []

img1 = np.float32(base_image)
img2 = np.float32(overlay_image)

# Calculate the average 
for i in range(len(base_image_points)):
    x = ( 1 - alpha ) * base_image_points[i][0] + alpha * overlay_image_points[i][0]
    y = ( 1 - alpha ) * base_image_points[i][1] + alpha * overlay_image_points[i][1]
    points.append((x,y))
    
imgMorph = np.zeros(img1.shape,img1.dtype)

with open(base_delaunay_points) as file:
    for line in file:
        x,y,z = line.split()
        x,y,z = int(x), int(y), int(z)
        t1 = [base_image_points[x], base_image_points[y], base_image_points[z]]
        t2 = [overlay_image_points[x], overlay_image_points[y], overlay_image_points[z]]
        t = [ points[x], points[y], points[z] ]
        
        morphTriangle(img1, img2, imgMorph, t1, t2, t, alpha)
        cv2.imshow("Animate Delaunay",np.uint8(imgMorph))
        cv2.waitKey(100)

cv2.imshow("Morphed Face", np.uint8(imgMorph))
cv2.waitKey()
cv2.destroyAllWindows()
